In [1]:
from shoestring.design import Design

import warnings

from Bio import BiopythonParserWarning
warnings.simplefilter('ignore', BiopythonParserWarning)

In [2]:
from shoestring.design import Design
from pyblast.utils import load_fasta_glob, load_genbank_glob, make_circular, make_linear
from shoestring.cost import SpanCost
from os.path import join

span_cost = SpanCost()


here = 'tests'
query = '*.gb'
paths = {
        'primers': join(here, "data/test_data/primers/primers.fasta"),
        'templates': join(here, "data/test_data/genbank/templates/*.gb"),
        'queries': join(
            'tests', "data/test_data/genbank/designs/pmodkan-ho-pact1-z4-er-vpr.gb"
        ),
    }
primers = make_linear(load_fasta_glob(paths["primers"]))
templates = load_genbank_glob(paths["templates"])

query_path = join(here, 'data/test_data/genbank/designs', query)
queries = make_circular(load_genbank_glob(query_path))

/Users/justinvrana/Documents/GitHub/shoestringDNA/shoestring/cost/__init__.py:132: RuntimeWarning: divide by zero encountered in true_divide
  self.cost_matrix = (m * CostParams.material + t * CostParams.time) * 1.0 / e


In [3]:
design = Design(span_cost=span_cost)

design.add_materials(primers=primers, templates=templates, queries=queries)

design.compile()

df = design.design()

df

Found previous installation at "/Users/justinvrana/anaconda3/lib/python3.7/site-packages/pyblast/blast_bin/bin/ncbi-blast-2.9.0+/bin"
CMD: makeblastdb -dbtype nucl -title a0cc813f-660e-44b5-8b8e-a932d47ac9ec -out /var/folders/fr/yc9x0z2s39lcj4f9d95wvrpr0000gn/T/tmpths27saf/a0cc813f-660e-44b5-8b8e-a932d47ac9ec -in /var/folders/fr/yc9x0z2s39lcj4f9d95wvrpr0000gn/T/tmpa7dr1_f2.fasta
CMD: blastn -db /var/folders/fr/yc9x0z2s39lcj4f9d95wvrpr0000gn/T/tmpths27saf/a0cc813f-660e-44b5-8b8e-a932d47ac9ec -out /var/folders/fr/yc9x0z2s39lcj4f9d95wvrpr0000gn/T/tmpths27saf/tmpuo3ynul6 -query /var/folders/fr/yc9x0z2s39lcj4f9d95wvrpr0000gn/T/tmpih2hqp6e.fasta -outfmt "7 qacc sacc score evalue bitscore length nident gapopen gaps qlen qstart qend slen sstart send sstrand qseq sseq"
CMD: makeblastdb -dbtype nucl -title d0c40f6d-b4cf-4313-aced-db3cd6fe4b9f -out /var/folders/fr/yc9x0z2s39lcj4f9d95wvrpr0000gn/T/tmpu8hju3y7/d0c40f6d-b4cf-4313-aced-db3cd6fe4b9f -in /var/folders/fr/yc9x0z2s39lcj4f9d95wvrpr0000gn/T

,anneal_seq,cost,fragment_length,fragment_seq,query,query_name,query_region,subject,subject_name,subject_region,type
0,NaN,90,674,GGTACCTTTAAGACCAATGACTTACAAGGCAGCTGTAGATCTTAGC...,5c4d5254-c72f-46ef-ab18-01507e852368,pLKO-pEF1a-FRT-tdTomato,"(734, 1408)",f1f8a4d8-3f6c-4bd6-aa21-72641b82c455,pCW57.1,"(4815, 5489)",PCR_PRODUCT
1,GGTACCTTTAAGACCAATGA,?,0,NaN,5c4d5254-c72f-46ef-ab18-01507e852368,pLKO-pEF1a-FRT-tdTomato,"(734, 1408)",f1f8a4d8-3f6c-4bd6-aa21-72641b82c455,pCW57.1,"(4815, 4835)",PRIMER
2,AGGCCTCCAAAAAAGCCTCC,?,0,NaN,5c4d5254-c72f-46ef-ab18-01507e852368,pLKO-pEF1a-FRT-tdTomato,"(734, 1408)",f1f8a4d8-3f6c-4bd6-aa21-72641b82c455,pCW57.1,"(5469, 5489)",PRIMER
3,NaN,456.333,686,AGGGACGTACCCAATTCGCCCTATAGTGAGTCGTATTACGCGCGCT...,5c4d5254-c72f-46ef-ab18-01507e852368,pLKO-pEF1a-FRT-tdTomato,"(1408, 2094)",None,SYNTHESIS,None,JUNCTION_BY_SYNTHESIS
4,NaN,90,1060,GATTATCAAAAAGGATCTTCACCTAGATCCTTTTAAATTAAAAATG...,5c4d5254-c72f-46ef-ab18-01507e852368,pLKO-pEF1a-FRT-tdTomato,"(2094, 3154)",ec03a8f1-f83e-43df-ab34-36e1c6c04b30,bla_CDA,"(0, 1060)",PCR_PRODUCT
5,GATTATCAAAAAGGATCTTC,?,0,NaN,5c4d5254-c72f-46ef-ab18-01507e852368,pLKO-pEF1a-FRT-tdTomato,"(2094, 3154)",ec03a8f1-f83e-43df-ab34-36e1c6c04b30,bla_CDA,"(0, 20)",PRIMER
6,CGCGGAACCCCTATTTGTTT,?,0,NaN,5c4d5254-c72f-46ef-ab18-01507e852368,pLKO-pEF1a-FRT-tdTomato,"(2094, 3154)",ec03a8f1-f83e-43df-ab34-36e1c6c04b30,bla_CDA,"(1040, 1060)",PRIMER
7,NaN,122,54,TCATGACCAAAATCCCTTAACGTGAGTTTTCGTTCCACTGAGCGTC...,5c4d5254-c72f-46ef-ab18-01507e852368,pLKO-pEF1a-FRT-tdTomato,"(3154, 3208)",None,SYNTHESIS,None,JUNCTION_BY_PRIMERS
8,NaN,90,629,GGCCGCGTTGCTGGCGTTTTTCCATAGGCTCCGCCCCCCTGACGAG...,5c4d5254-c72f-46ef-ab18-01507e852368,pLKO-pEF1a-FRT-tdTomato,"(3208, 3837)",291ab9ce-c9a2-4894-8700-9f1283b51e7a,attB-mCh-Dnmt3b1-Poly(A,"(6488, 7117)",PCR_PRODUCT
9,GGCCGCGTTGCTGGCGTTTT,?,0,NaN,5c4d5254-c72f-46ef-ab18-01507e852368,pLKO-pEF1a-FRT-tdTomato,"(3208, 3837)",291ab9ce-c9a2-4894-8700-9f1283b51e7a,attB-mCh-Dnmt3b1-Poly(A,"(6488, 6508)",PRIMER
